# Problem :
Current dataset prepared for benchmark filtering ('small_dataset.xlsx') is too small (1000 rows) to consider adding to benchmark filtering another criterion which is Fill Rate. Secondly, this dataset does not have the feature Fill Rate and adding it is not possible.

# Solution :
1. Use a current small dataset as a source for creating a large dataset by applying a bootstrap sampling approach.
2. Generate Fill Rate by applying Monte Carlo simulation.

In [7]:
import pandas as pd
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [8]:
#import os
#import sys

#os.chdir('C:/Projects/Metric Calculator')
#sys.path.append('C:/Projects')

In [9]:
from constants import (
    VOIVODSHIP_LIST,
    INDUSTRY_LIST,
    JOB_TYPE_LIST
)

In [10]:
#Importing the source dataset

input_df = pd.read_excel('Datasets/small_dataset.xlsx')
input_df.head(10)

,Voivodship,Industry,Job Type,Salary,Bonus,Benefits,Contract Length,Contract Type,Brand Recognition,Recruitment Difficulty,Shifts,Overtime,Employee Satisfaction
0,Małopolskie,Warehousing,"Machine operators, fitters, etc.",5462.52,978.73,One benefit,2,Contract of employment,Weak,Normal,Two 8hrs shifts,No overtime,Rather high
1,Świętokrzyskie,Manufacturing,"Specialists, e.g. turners, welders, electricia...",10197.17,1738.25,Two benefits,12,Mandate contract,Weak,Easy,One 8hrs shift,No overtime,Rather low
2,Zachodniopomorskie,Manufacturing,"Production workers, production operator, etc.",7568.14,818.59,One benefit,19,Contract of employment,Moderate,Normal,Two 8hrs shifts,No overtime,Rather high
3,Warmińsko-Mazurskie,Others,"Warehousment, warehouse workers, etc.",8825.89,2502.22,Two benefits,10,Contract work,Very weak,Very easy,Two 12hrs shifts,Ocassional overtime,Very low
4,Śląskie,Warehousing,"Packers, pickers, sorters, etc.",6782.31,1786.56,Two benefits,5,Contract of employment,Strong,Very difficult,Three 8hrs shifts,Ocassional overtime,Moderate
5,Warmińsko-Mazurskie,Manufacturing,"Forklift operators, internal logistics operato...",10622.78,120.89,One benefit,22,Mandate contract,Very strong,Difficult,One 8hrs shift,Regular overtime,Very low
6,Małopolskie,Manufacturing,"Specialists, e.g. turners, welders, electricia...",12258.43,3473.15,Two benefits,20,Contract of employment,Strong,Normal,Three 8hrs shifts,Ocassional overtime,Very low
7,Podlaskie,Others,"Production workers, production operator, etc.",5136.13,742.91,Two benefits,16,Contract of employment,Strong,Very difficult,Two 12hrs shifts,Ocassional overtime,Rather high
8,Dolnośląskie,Manufacturing,"Packers, pickers, sorters, etc.",6188.93,606.54,Two benefits,1,Contract of employment,Moderate,Easy,Three 8hrs shifts,Ocassional overtime,Rather high
9,Łódzkie,Manufacturing,"Machine operators, fitters, etc.",9337.03,944.49,Two benefits,17,Contract of employment,Strong,Normal,Two 12hrs shifts,No overtime,Moderate


In [11]:
#Eliminating space in a 'Job Type' column (this column is a filtering key)

input_df.rename(columns = {'Job Type' : 'Job_Type'}, inplace = True)
input_df.head(3)

,Voivodship,Industry,Job_Type,Salary,Bonus,Benefits,Contract Length,Contract Type,Brand Recognition,Recruitment Difficulty,Shifts,Overtime,Employee Satisfaction
0,Małopolskie,Warehousing,"Machine operators, fitters, etc.",5462.52,978.73,One benefit,2,Contract of employment,Weak,Normal,Two 8hrs shifts,No overtime,Rather high
1,Świętokrzyskie,Manufacturing,"Specialists, e.g. turners, welders, electricia...",10197.17,1738.25,Two benefits,12,Mandate contract,Weak,Easy,One 8hrs shift,No overtime,Rather low
2,Zachodniopomorskie,Manufacturing,"Production workers, production operator, etc.",7568.14,818.59,One benefit,19,Contract of employment,Moderate,Normal,Two 8hrs shifts,No overtime,Rather high


In [12]:
#Creating a large dataset using bootstrap sampling

n = 30000 #number of rows declared for bootstrap generation - keep in mind that some filters combination may not return any rows for bootstrap (and the loop will pass) 

#Creating an empty bootstrap dataframe
output_df = pd.DataFrame(columns = input_df.columns)

#Loop for populating the bootstrap dataframe
for i in range(0, n) :
    
    #Creating a filtering criterions for selecting rows from which a single bootstrap row will be generated
    filtering_list = []

    filtering_list.append(random.choice(VOIVODSHIP_LIST))
    filtering_list.append(random.choice(INDUSTRY_LIST))
    filtering_list.append(random.choice(JOB_TYPE_LIST))
    
    voivodship = filtering_list[0]
    industry = filtering_list[1]
    job_type = filtering_list[2]
    
    #Checking whether filtering criterions will return any rows
    if len(input_df.query('\
           Voivodship == @voivodship and \
           Industry == @industry and \
           Job_Type == @job_type \
           ')) > 0 :
        
        #Selecting rows for particular bootstrap sample row
        df_filter = input_df.query('\
                    Voivodship == @voivodship and \
                    Industry == @industry and \
                    Job_Type == @job_type \
                    ')
        
        #Creating a single row via bootstrap sampling
        new_row = {col: np.random.choice(df_filter[col]) for col in df_filter.columns}
        
        #Adding newly created record to the bootstrap dataframe
        output_df = output_df.append(new_row, ignore_index = True)

In [14]:
len(output_df) # --> 26420 rows will do (c. 90% of declared procedures generated a bootstrap row)

26420

BELOW : It has been agreed with the business representative that particular job types are more likely to yield 'Fill Rate' from specific intervals, like those below. Those intervals (delivered by the business representative) will be used in Monte Carlo simulation (dictionary job_type_proba_intervals_narrow).

CAVEAT : Applying these intervals in Monte Carlo simulation will only generate 'Fill Rate' from these ranges; whilst some of real cases would be outside of them. To fix that issue a second dictionary is created where all job types fall into [0, 1] interval (dictionary job_type_proba_intervals_wide). Every time a dictionary will be randomly choosen.

In [15]:
job_type_proba_intervals_narrow = {
    "Forklift operators, internal logistics operators, etc." : [0.60, 0.75],
    "Machine operators, fitters, etc." : [0.70, 0.85],
    "Packers, pickers, sorters, etc." : [0.75, 0.90],
    "Production workers, production operator, etc." : [0.75, 0.95],
    "Specialists, e.g. turners, welders, electricians etc." : [0.55, 0.70],
    "Warehousment, warehouse workers, etc." : [0.65, 0.80]
}

In [16]:
job_type_proba_intervals_wide = {
    "Forklift operators, internal logistics operators, etc." : [0, 1],
    "Machine operators, fitters, etc." : [0, 1],
    "Packers, pickers, sorters, etc." : [0, 1],
    "Production workers, production operator, etc." : [0, 1],
    "Specialists, e.g. turners, welders, electricians etc." : [0, 1],
    "Warehousment, warehouse workers, etc." : [0, 1]
}

In [17]:
#Adding an ampty Fill Rate column

output_df['Fill_Rate'] = ""
output_df.head(3)

,Voivodship,Industry,Job_Type,Salary,Bonus,Benefits,Contract Length,Contract Type,Brand Recognition,Recruitment Difficulty,Shifts,Overtime,Employee Satisfaction,Fill_Rate
0,Lubelskie,Warehousing,"Warehousment, warehouse workers, etc.",6635.57,432.67,No benefits,18,Contract of employment,Strong,Easy,Three 8hrs shifts,Ocassional overtime,Very high,
1,Zachodniopomorskie,Manufacturing,"Warehousment, warehouse workers, etc.",7010.34,164.83,One benefit,1,Contract of employment,Weak,Very difficult,One 8hrs shift,Ocassional overtime,Rather high,
2,Śląskie,Others,"Specialists, e.g. turners, welders, electricia...",10376.41,2357.67,Two benefits,11,Contract of employment,Very strong,Normal,Two 8hrs shifts,Ocassional overtime,Very low,


In [18]:
#Populating Fill Rate column using Monte Carlo simulation

for i in range(len(output_df)) :
    
    for j in range(len(JOB_TYPE_LIST)) :
        
        if output_df['Job_Type'].loc[i] == JOB_TYPE_LIST[j] :
            
            dict_choice = random.choice(['narrow', 'wide'])
            
            if  dict_choice == 'narrow' :
                
                #Converting dictionary wit NARROW intervals to a list of tuples, each tuple containing (job_type, lower, upper)
                data = [(job_type, proba[0], proba[1]) for job_type, proba in job_type_proba_intervals_narrow.items()]

                #Creating a DataFrame with probability bounds
                proba_df = pd.DataFrame(data, columns = ['Job_Type', 'Lower', 'Upper'])
            
            else :
            
                #Converting dictionary with WIDE to a list of tuples, each tuple containing (job_type, lower, upper)
                data = [(job_type, proba[0], proba[1]) for job_type, proba in job_type_proba_intervals_wide.items()]

                #Creating a DataFrame with probability bounds
                proba_df = pd.DataFrame(data, columns = ['Job_Type', 'Lower', 'Upper'])
            
            output_df['Fill_Rate'].loc[i] = random.uniform(proba_df['Lower'].loc[j], proba_df['Upper'].loc[j])

In [20]:
output_df.head(10)

,Voivodship,Industry,Job_Type,Salary,Bonus,Benefits,Contract Length,Contract Type,Brand Recognition,Recruitment Difficulty,Shifts,Overtime,Employee Satisfaction,Fill_Rate
0,Lubelskie,Warehousing,"Warehousment, warehouse workers, etc.",6635.57,432.67,No benefits,18,Contract of employment,Strong,Easy,Three 8hrs shifts,Ocassional overtime,Very high,0.794312
1,Zachodniopomorskie,Manufacturing,"Warehousment, warehouse workers, etc.",7010.34,164.83,One benefit,1,Contract of employment,Weak,Very difficult,One 8hrs shift,Ocassional overtime,Rather high,0.695328
2,Śląskie,Others,"Specialists, e.g. turners, welders, electricia...",10376.41,2357.67,Two benefits,11,Contract of employment,Very strong,Normal,Two 8hrs shifts,Ocassional overtime,Very low,0.585503
3,Zachodniopomorskie,Others,"Specialists, e.g. turners, welders, electricia...",5918.88,2026.99,One benefit,4,Contract work,Weak,Difficult,One 8hrs shift,No overtime,Rather high,0.50554
4,Śląskie,Warehousing,"Machine operators, fitters, etc.",5615.66,1493.48,Two benefits,14,Contract of employment,Very strong,Easy,Two 12hrs shifts,Ocassional overtime,Rather high,0.851513
5,Mazowieckie,Manufacturing,"Machine operators, fitters, etc.",8606.22,2566.72,Two benefits,19,Contract of employment,Very strong,Easy,Two 12hrs shifts,Ocassional overtime,Moderate,0.83467
6,Świętokrzyskie,Manufacturing,"Forklift operators, internal logistics operato...",2945.00,536.95,One benefit,24,Contract of employment,Very strong,Difficult,Three 8hrs shifts,No overtime,Moderate,0.081316
7,Zachodniopomorskie,Warehousing,"Warehousment, warehouse workers, etc.",6754.13,664.32,All benefits,13,Contract of employment,Strong,Normal,Two 12hrs shifts,Regular overtime,Very high,0.765995
8,Małopolskie,Others,"Packers, pickers, sorters, etc.",5282.06,927.64,All benefits,3,Contract work,Very strong,Easy,Two 12hrs shifts,Regular overtime,Rather high,0.214569
9,Dolnośląskie,Warehousing,"Machine operators, fitters, etc.",8819.52,636.74,Two benefits,23,Contract of employment,Strong,Difficult,Three 8hrs shifts,No overtime,Moderate,0.404269


In [21]:
output_df.to_excel('Datasets/large_dataset.xlsx', index = False)